In [3]:
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader

### create model class

In [4]:
class NeuralNetwork(torch.nn.Module):
    # parameterize num of inputs & outputs to reuse same code for diff datasets with diff num of features and classes
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # first hidden layer
            # Linear layer takes num of input and output nodes as args
            torch.nn.Linear(num_inputs, 30),
            # Nonlinear activation functions are placed bw hidden layers
            torch.nn.ReLU(),

            # second hidden layer
            # The num of output nodes in prev hidden layer is equal to input of next hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )
    
    def forward(self, x):
        logits = self.layers(x)
        # the output of the last layer are called logits
        return logits

### create ToyDataset

In [5]:
# create Toy dataset
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0,1])

In [6]:
# create custome dataset class
class ToyDataset(Dataset):
    def __init__(self, X, y):
        # these vars could be file paths, file objects, database connectors, etc. we are using X,y bc created toydataset above sitting in mem
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        # retrieve exactly one data record and corresponding label
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        # return dataset row len
        return self.labels.shape[0]

In [7]:
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

### create DataLoader

In [8]:
torch.manual_seed(123)

In [27]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    drop_last=True
)

### train NN model

In [28]:
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.5
)

In [29]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
            f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
            f" | Train Loss: {loss:.2f}")

Epoch: 001/003 | Batch 000/002 | Train Loss: 0.74
Epoch: 001/003 | Batch 001/002 | Train Loss: 0.58
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.40
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.56
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.05
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.04


In [32]:
model.eval()

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [31]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of model parameters = ", num_params)

Total number of model parameters =  752


### predictions

In [ ]:
# logits output
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 1.9280, -2.0601],
        [ 1.7927, -1.9219],
        [ 1.5995, -1.7219],
        [-2.5382,  2.0873],
        [-3.0131,  2.4429]])


In [ ]:
# class membership probabilites
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

# first pred has 0.9818, or 98.18%, probability of belong to class 0 and 0.0182, or 1.82%, probability of belong to class 1.

tensor([[0.9818, 0.0182],
        [0.9762, 0.0238],
        [0.9652, 0.0348],
        [0.0097, 0.9903],
        [0.0043, 0.9957]])


In [36]:
# class prediction labels from class membership probabilities using argmax
probas_predictions = torch.argmax(probas, dim=1)
print(probas_predictions)

tensor([0, 0, 0, 1, 1])


In [37]:
# class prediction labels from logits using argmax
logit_predicitions = torch.argmax(outputs, dim=1)
print(logit_predicitions)

tensor([0, 0, 0, 1, 1])


In [39]:
# compare preds to test dataset
logit_predicitions == y_train

tensor([True, True, True, True, True])

In [ ]:
# correct predictions count
torch.sum(logit_predicitions == y_train)

tensor(5)


In [45]:
# prediction accuracy function
def compute_accuracy(model, dataloader):
    """Function to compute prediction accuracy of pytorch model

    Args:
        model (_type_): Neural Network pytorch class
        dataloader (_type_): Dataloader pytorch class
    """

    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item()

In [44]:
compute_accuracy(model, train_loader)

1.0

In [46]:
compute_accuracy(model, test_loader)

1.0

### save & load model

In [ ]:
# save model
torch.save(model.state_dict(), "model.pth")

In [48]:
# load model
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>